# pt-BR | Web scraping - Beautiful Soup
> "Beautiful Soup & Python."
- toc: false
- branch: master
- badges: true
- comments: false
- categories: [fastpages, jupyter]
- hide: false
- author: leticiaplang

Seguindo com o **web scraping**, teremos nesse notebook a coleta de dados utilizando o **Beautiful Soup**. Diferente o Selenium, não utilizaremos janela de web para realizar o processo, faremos isso às cegas por meio de comendos.

### Web scraping com Beautiful Soup e Python

Para inciarmos o processo manteremos a realização de todas as instalações ou importações das bibliotecas ou pacotes que iremos utilizar. 
* **bs4** é a biblioteca que contém o beautiful soup. A característica dessa função é varrer todos os dados da página web (HTML), identificando grupos que contenham as informações que queremos. 
* **Pandas** é essencial para transformarmos os dados em estrutura de data frame, que possui mesma estrutura de uma tabela (clolunas e linhas).


In [ ]:
# Importações
import time  #se necessário utilizar um tempo de espera para rodar as próximas células
from bs4 import BeautifulSoup #biblioteca que contém módulo que utilizaremos
import pandas as pd #dataframe e exploração dos dados

Nesse artigo já abordarei o web scraping automatizado para rodar até n páginas definidas por mim. Poderíamos ter determinado esse valor automaticamente, retirando informações do próprio site, mas por hora abordarei deste modo para não dificultar a compreensão.

Achou muito confuso? Não se preocupe que irei abordar etapa por etapa desse código 😃 

In [ ]:
df_orig = pd.DataFrame(columns=['product', 
                               'area', 
                               'room', 
                               'bath', 
                               'garage', 
                               'adress', 
                               'price'])
page = 1

while page <=3:
    # Viva Real homepage
    url = f'https://www.imoveis.com.br/aluguel/sao-paulo/saopaulo/?pagina={page}'
    time.sleep(3)    
    # Usaremos o request para realizar a requisição
    vr_response = requests.get(url)
    time.sleep(2)
    # Conversão da resposta para HTML Usando Beautiful Soup
    vr_soup = BeautifulSoup(vr_response.text, 'html.parser')
    time.sleep(2)
    prod_lists = [product.text.lower() for product in vr_soup.find_all('span', {'class':'property-card__title'})]
    area_lists = [area.text.lower() for area in vr_soup.find_all('li', {'class':'property-card__detail-area'})]
    room_lists = [room.text.lower() for room in vr_soup.find_all('li', {'class':'property-card__detail-room'})]
    bath_lists = [bath.text.lower() for bath in vr_soup.find_all('li', {'class': 'property-card__detail-bathroom'})]
    garage_lists = [garage.text.lower() for garage in vr_soup.find_all('li', {'class':'property-card__detail-garage'})]
    adress_lists = [adress.text.lower() for adress in vr_soup.find_all('span', {'class':'property-card__address'})]
    price_lists = [price.text.lower() for price in vr_soup.find_all('div', {'class':'property-card__price'})]
    page += 1
    df_orig = df_orig.append(pd.DataFrame({'product':prod_lists, 
                                           'area':area_lists, 
                                           'room':room_lists, 
                                           'bath':bath_lists, 
                                           'garage':garage_lists, 
                                           'adress':adress_lists, 
                                           'price':price_lists}))

Quando utilizamos o **while** precisamos ter um limite que se torne falso para que você não entre em um looping infinito dos dados. Nesse caso, defino o número de páginas que vou querer analisar, iniciando pela página 1. Perceba que a página está fora do loop while, senão ela iria sempre reiniciar na 1 a cada vez que um novo ciclo se iniciasse, caindo no tal loop infinito.

Dentro do while defino a página que é de meu interesse realizar o web scraping. Nessa situação inserimos a **página que está com os dados que queremos coletar**, ou seja, se eu quiser buscar imóveis de São Paulo, preciso previamente ir no site, definir os critérios de busca para então saber qual é a URL que irei utilizar para a coleta com b4s.

Após isso, devemos analizar a URL do site. Clique para ir a próxima página e identifique **como a URL se estrutura**. Essa é a que iremos inserir no loop para que possamos estar fazendo o scrapping de todas as páginas do site.

No local da URL que possui o número dois (segunda página) iremos trocar por chaves que componham a variável página que definimos previamente. 

Após definida a URL, precisamos solicitar acesso para pegarmos os dados. Salvamos em uma variável, pois a resposta conterá informações do site que estamos buscando. Por tal motivo é que extraímos o texto dessa resposta para utilizarmos o "buscador de informações" do b4s chamado html.parser nesse caso.


In [ ]:
page = 1

while page <=3:
    # Imóveis homepage
    url = f'https://www.imoveis.com.br/aluguel/sao-paulo/saopaulo/?pagina={page}'
    time.sleep(3)    
    # Usaremos o request para realizar a requisição
    vr_response = requests.get(url)
    time.sleep(2)
     # Conversão da resposta para HTML Usando Beautiful Soup
    vr_soup = BeautifulSoup(vr_response.text, 'html.parser')
    time.sleep(2)

Tendo a URL definida com os dados que queremos, precisamos então descobrir quais são os elementos que buscamos na página. Você lembra como se faz? 🤔

No web scraping parte 1 abordei como podemos realizar a inspeção do elemento da página web e encontrar os elementos que caracterizem o dado que buscamos. Maaas, spoiler alert, não há fórmula perfeita, você precisará colocar a mão na massa e descobrir por tentativa e erro.

Iremos aqui trabalhar com **list comprehensions** 😎 e a lógica da construção é a seguinte:
* Definimos qual o output que queremos
* Definimos um for loop para que possamos passar dado a dado e transformar adequadamente cada um deles
* Inserimos dentro de dois colchetes que definem que o outpur será uma lista de valores.

Aqui defino como output cada nome de produto será texto minúsculo. 

O for loop irá procurar cada valor dentro dos valores que o "buscador de valores" do b4s encontrou no site que fecham os os identificadores que defini. 
  
*vr_soup* é a variável que contém os dados HTML do site e o buscador

*find_all* define que queremos que o "buscador" traga todos os itens que possuem a característica de definimos

* a característica é definida por: (grande grupo(elemento que identifica determinada informação dentro do grande grupo))
* Perceba que difrentemente do selenium, devemos informar que é uma classe para depois informar a designação do delemento.

In [ ]:
    # Definindo a construção das listas para cada tipo de informação
    prod_lists = [product.text.lower() for product in vr_soup.find_all('span', {'class':'property-card__title'})]
    area_lists = [area.text.lower() for area in vr_soup.find_all('li', {'class':'property-card__detail-area'})]
    room_lists = [room.text.lower() for room in vr_soup.find_all('li', {'class':'property-card__detail-room'})]
    bath_lists = [bath.text.lower() for bath in vr_soup.find_all('li', {'class': 'property-card__detail-bathroom'})]
    garage_lists = [garage.text.lower() for garage in vr_soup.find_all('li', {'class':'property-card__detail-garage'})]
    adress_lists = [adress.text.lower() for adress in vr_soup.find_all('span', {'class':'property-card__address'})]
    price_lists = [price.text.lower() for price in vr_soup.find_all('div', {'class':'property-card__price'})]

Na parte final do código, ainda dentro do while loop, iremos inserir o código que fará o valor da página trocar e iremos adicionar ao dataframe os valores/informações retiradas.

Antes de iniciarmos o while loop, devemos criar a estrtura do data frame/tabela que queremos ter como resultado, definindo apenas as colunas.

In [ ]:
# Definindo a estrtura do data frame fora do while loop
df = pd.DataFrame(columns=['product', 
                           'area', 
                           'room', 
                           'bath', 
                           'garage', 
                           'adress', 
                           'price'])

Tendo este data frame criado, é possível inserir novas informações nele por meio do uso do 'append', em formato de DF contendo em um dicionário coluna : lista de valores.

Além disso teremos como definição do número da página, o valor da página daquele loop + 1. Quando a página for igual a 4, o while não irá continuar, uma vez que o retorno será False por não ser menor ou igual a 3.

In [ ]:
    page += 1
    df = df.append(pd.DataFrame({'product':prod_lists, 
                                 'area':area_lists, 
                                 'room':room_lists, 
                                 'bath':bath_lists, 
                                 'garage':garage_lists, 
                                 'adress':adress_lists, 
                                 'price':price_lists}))

Saindo do while loop, podemos verificar as informações do DF e exportá-lo para a máquina local

In [ ]:
df.head() #verificar as 5 primeiras linhas
df.shape() #verificar número de linhas e colunas 
df.duplicated() #verificar se há valores duplicados

In [ ]:
df.to_csv('imoveis.csv') #exportar df para máquina local